# Install feature data and modules

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import optuna
import xgboost as xgb
from sklearn.metrics import *
import numpy as np
from xgboost import XGBClassifier, DMatrix, train as xgb_train


# Import data

In [2]:
data = pd.read_csv(r'/media/minhman/01DA5FE5725E67B0/Code_HK6/NMTGMT/densenet_features_with_filenames.csv')

In [3]:
data.head(1)

,0,1,2,3,4,5,6,7,8,9,...,1912,1913,1914,1915,1916,1917,1918,1919,label,filename
0,0.000077,0.001443,0.000719,0.001873,0.000745,0.000065,0.000515,0.000278,0.000109,0.015095,...,0.581535,0.343685,0.408467,0.162864,0.63842,0.2912,0.053712,0.259736,1,28908.jpg


In [4]:
X_data = data.iloc[:, :-2]
y_data = data.iloc[:, [-2, -1]]

In [5]:
X_data.head(1)

,0,1,2,3,4,5,6,7,8,9,...,1910,1911,1912,1913,1914,1915,1916,1917,1918,1919
0,0.000077,0.001443,0.000719,0.001873,0.000745,0.000065,0.000515,0.000278,0.000109,0.015095,...,0.066967,2.389044,0.581535,0.343685,0.408467,0.162864,0.63842,0.2912,0.053712,0.259736


In [6]:
y_data.head(1)

,label,filename
0,1,28908.jpg


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.3, random_state=42)
y_train = y_train.iloc[:, [0]]
X_train, X_valid, y_train , y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

# XGBoost


In [8]:
# def objective(trial):
#     params = {
#         'objective': 'multi:softprob',
#         'eval_metric': 'mlogloss',
#         'num_class': 3,
#         'verbosity': 0,
#         'tree_method': 'gpu_hist',
#         'predictor': 'gpu_predictor',
#         'grow_policy': 'lossguide',
#         'max_bin': 128,
#         'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
#         'max_depth': trial.suggest_int('max_depth', 2, 6),  # Giảm
#         'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
#         'subsample': trial.suggest_float('subsample', 0.5, 1.0),
#         'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
#         'lambda': trial.suggest_float('lambda', 1e-8, 10.0, log=True),
#         'alpha': trial.suggest_float('alpha', 1e-8, 10.0, log=True),
#         'seed': 42,
#     }

#     dtrain = xgb.DMatrix(X_train, label=y_train)
#     dvalid = xgb.DMatrix(X_valid, label=y_valid)

#     model = xgb.train(
#         params,
#         dtrain,
#         num_boost_round=100,  # Giảm từ 200 xuống
#         evals=[(dvalid, 'validation')],
#         early_stopping_rounds=10,
#         verbose_eval=False
#     )

#     preds = model.predict(dvalid)
#     preds_labels = np.argmax(preds, axis=1)
#     acc = accuracy_score(y_valid, preds_labels)
#     return acc


# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=50)

# print("Best trial:")
# print(study.best_trial)
# print("Best parameters:")
# print(study.best_params)


In [9]:
# Train lại mô hình với tham số tốt nhất
best_model = XGBClassifier({'learning_rate': 0.07377159347911756, 'max_depth': 5, 'min_child_weight': 1, 'subsample': 0.8386473998722536, 'colsample_bytree': 0.673677423485615, 'lambda': 0.10834487137077414, 'alpha': 8.021854676733537e-05})
best_model.fit(X_train, y_train)

/home/minhman/.local/lib/python3.10/site-packages/xgboost/core.py:726: FutureWarning: Pass `objective` as keyword args.
  warnings.warn(msg, FutureWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

In [12]:
y_pred = best_model.predict(X_test)

In [14]:
# CatBooost
y_pred_xgb= best_model.predict(X_test)
report_xgb = classification_report(y_test.iloc[:, [0]], y_pred_xgb)

In [15]:
print(report_xgb)

              precision    recall  f1-score   support

           0       0.73      0.79      0.76      1092
           1       0.63      0.64      0.63      1110
           2       0.73      0.67      0.70       984

    accuracy                           0.70      3186
   macro avg       0.70      0.70      0.70      3186
weighted avg       0.70      0.70      0.70      3186



In [20]:
from sklearn.metrics import f1_score, accuracy_score

In [17]:
f1_score(y_test.iloc[:, [0]], y_pred_xgb, average='micro')

0.6977401129943502

In [18]:
f1_score(y_test.iloc[:, [0]], y_pred_xgb, average='macro')

0.6975139933148956

In [22]:
accuracy_score(y_test.iloc[:, [0]], y_pred_xgb)

0.6977401129943502